# Applying ML to neurodevelopmental discorders detection

In [52]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

from problem import get_cv
from problem import get_all_data
from problem import split_data
from download_data import fetch_fmri_time_series

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.multiclass import unique_labels
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import roc_auc_score, accuracy_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.utils.multiclass import unique_labels

from nilearn.connectome import ConnectivityMeasure

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Loading data

In [93]:
data, labels = get_all_data()
data_train, labels_train, data_test, labels_test = split_data(data, labels)
data_abide=data[data['participants_dataset']=='abide']
data_adhd=data[data['participants_dataset']=='adhd200']
labels_adhd=labels[0:521]
labels_abide=labels[521:1671]

## Evaluations functions 

In [58]:
def evaluation(X, y):
    pipe = make_pipeline(FeatureExtractor(), Classifier())
    cv = get_cv(X, y)
    results = cross_validate(pipe, X, y, scoring=('roc_auc', 'accuracy'), cv=cv,
                             verbose=1, return_train_score=True,
                             n_jobs=1)
    
    return results

#Validation on new data
def evaluation_vv(X_train, y_train, X_test, y_test):
    pipe = make_pipeline(FeatureExtractor(), Classifier())
    clf = pipe.fit(X_train, y_train)
    y_pred_train = clf.predict(X_train)
    y_pred_test = clf.predict(X_test)
    accuracy_train = accuracy_score(y_train, y_pred_train)
    roc_auc_train = roc_auc_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    roc_auc_test = roc_auc_score(y_test, y_pred_test)
    return accuracy_train, roc_auc_train, accuracy_test, roc_auc_test

## Only anatomical features

In [59]:
class FeatureExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y):
        return self

    def transform(self, X_df):
        # get only the anatomical information
        X = X_df[[col for col in X_df.columns if col.startswith('anatomy')]]
        return X.drop(columns='anatomy_select')
    
class Classifier(BaseEstimator):
    def __init__(self):
        self.clf = make_pipeline(StandardScaler(), LogisticRegression(solver='lbfgs', max_iter=500))

    def fit(self, X, y):
        self.clf.fit(X, y)
        self.classes_ = unique_labels(y)
        return self
        
    def predict(self, X):
        return self.clf.predict(X)

    def predict_proba(self, X):
        return self.clf.predict_proba(X)

In [96]:
results_anat = evaluation(data, labels)
results_anat_abide = evaluation(data_abide, labels_abide)
results_anat_adhd = evaluation(data_adhd, labels_adhd)
results_anat_abide_adhd = evaluation_vv(data_abide, labels_abide, data_adhd, labels_adhd)
results_anat_adhd_abide = evaluation_vv(data_adhd, labels_adhd, data_abide, labels_abide)


print('Dataset              Train                  Test')
print('               ROC-AUC | Accuracy    ROC-AUC | Accuracy')
print("ADHD+ABIDE     {:.3f}     {:.3f}       {:.3f}     {:.3f}".format(np.mean(results_anat['train_roc_auc']), np.mean(results_anat['train_accuracy']), np.mean(results_anat['test_roc_auc']), np.mean(results_anat['test_accuracy'])))
print("ABIDE          {:.3f}     {:.3f}       {:.3f}     {:.3f}".format(np.mean(results_anat_abide['train_roc_auc']), np.mean(results_anat_abide['train_accuracy']), np.mean(results_anat_abide['test_roc_auc']), np.mean(results_anat_abide['test_accuracy'])))
print("ADHD           {:.3f}     {:.3f}       {:.3f}     {:.3f}".format(np.mean(results_anat_adhd['train_roc_auc']), np.mean(results_anat_adhd['train_accuracy']), np.mean(results_anat_adhd['test_roc_auc']), np.mean(results_anat_adhd['test_accuracy'])))
print("ABIDE on ADHD  {:.3f}     {:.3f}       {:.3f}     {:.3f}".format(np.mean(results_anat_abide_adhd[1]), np.mean(results_anat_abide_adhd[0]), np.mean(results_anat_abide_adhd[3]), np.mean(results_anat_abide_adhd[2])))
print("ADHD on ABIDE  {:.3f}     {:.3f}       {:.3f}     {:.3f}".format(np.mean(results_anat_adhd_abide[1]), np.mean(results_anat_adhd_abide[0]), np.mean(results_anat_adhd_abide[3]), np.mean(results_anat_adhd_abide[2])))


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Dataset              Train                  Test
               ROC-AUC | Accuracy    ROC-AUC | Accuracy
ADHD+ABIDE     0.800     0.726       0.661     0.632
ABIDE          0.763     0.700       0.495     0.486
ADHD           0.942     0.873       0.621     0.606
ABIDE on ADHD  0.675     0.677       0.464     0.482
ADHD on ABIDE  0.808     0.831       0.473     0.486


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.4s finished


## fMRI-derived features

In [41]:
def _load_fmri(fmri_filenames):
    """Load time-series extracted from the fMRI using a specific atlas."""
    return np.array([pd.read_csv(subject_filename,
                                 header=None).values
                     for subject_filename in fmri_filenames])


class FeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        # make a transformer which will load the time series and compute the
        # connectome matrix
        self.transformer_fmri = make_pipeline(
            FunctionTransformer(func=_load_fmri, validate=False),
            ConnectivityMeasure(kind='tangent', vectorize=True))
        
    def fit(self, X_df, y):
        # get only the time series for the MSDL atlas
        fmri_filenames = X_df['fmri_msdl']
        self.transformer_fmri.fit(fmri_filenames, y)
        return self

    def transform(self, X_df):
        fmri_filenames = X_df['fmri_msdl']
        return self.transformer_fmri.transform(fmri_filenames)

class Classifier(BaseEstimator):
    def __init__(self):
        self.clf = make_pipeline(StandardScaler(), LogisticRegression(C=1.))

    def fit(self, X, y):
        self.clf.fit(X, y)
        self.classes_ = unique_labels(y)
        return self
       
    def predict(self, X):
        return self.clf.predict(X)

    def predict_proba(self, X):
        return self.clf.predict_proba(X)

In [42]:
results_anat = evaluation(data, labels)
results_anat_abide = evaluation(data_abide, labels_abide)
results_anat_adhd = evaluation(data_adhd, labels_adhd)
results_anat_abide_adhd = evaluation_vv(data_abide, labels_abide, data_adhd, labels_adhd)
results_anat_adhd_abide = evaluation_vv(data_adhd, labels_adhd, data_abide, labels_abide)


print('Dataset              Train                  Test')
print('               ROC-AUC | Accuracy    ROC-AUC | Accuracy')
print("ADHD+ABIDE     {:.3f}     {:.3f}       {:.3f}     {:.3f}".format(np.mean(results_anat['train_roc_auc']), np.mean(results_anat['train_accuracy']), np.mean(results_anat['test_roc_auc']), np.mean(results_anat['test_accuracy'])))
print("ABIDE          {:.3f}     {:.3f}       {:.3f}     {:.3f}".format(np.mean(results_anat_abide['train_roc_auc']), np.mean(results_anat_abide['train_accuracy']), np.mean(results_anat_abide['test_roc_auc']), np.mean(results_anat_abide['test_accuracy'])))
print("ADHD           {:.3f}     {:.3f}       {:.3f}     {:.3f}".format(np.mean(results_anat_adhd['train_roc_auc']), np.mean(results_anat_adhd['train_accuracy']), np.mean(results_anat_adhd['test_roc_auc']), np.mean(results_anat_adhd['test_accuracy'])))
print("ABIDE on ADHD  {:.3f}     {:.3f}       {:.3f}     {:.3f}".format(np.mean(results_anat_abide_adhd[1]), np.mean(results_anat_abide_adhd[0]), np.mean(results_anat_abide_adhd[3]), np.mean(results_anat_abide_adhd[2])))
print("ADHD on ABIDE  {:.3f}     {:.3f}       {:.3f}     {:.3f}".format(np.mean(results_anat_adhd_abide[1]), np.mean(results_anat_adhd_abide[0]), np.mean(results_anat_adhd_abide[3]), np.mean(results_anat_adhd_abide[2])))


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  3.1min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  2.2min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:   58.3s finished


                     Train                  Test
               ROC-AUC | Accuracy    ROC-AUC | Accuracy
ADHD+ABIDE     1.000     1.000       0.596     0.596
ABIDE          1.000     1.000       0.612     0.612
ADHD           1.000     1.000       0.623     0.623
ABIDE on ADHD  1.000     1.000       0.571     0.619
ADHD on ABIDE  1.000     1.000       0.520     0.530


## Combining anatomy and fMRI

In [46]:
def _load_fmri(fmri_filenames):
    """Load time-series extracted from the fMRI using a specific atlas."""
    return np.array([pd.read_csv(subject_filename,
                                 header=None).values
                     for subject_filename in fmri_filenames])


class FeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        # make a transformer which will load the time series and compute the
        # connectome matrix
        self.transformer_fmri = make_pipeline(
            FunctionTransformer(func=_load_fmri, validate=False),
            ConnectivityMeasure(kind='tangent', vectorize=True))
    
    def fit(self, X_df, y):
        fmri_filenames = X_df['fmri_msdl']
        self.transformer_fmri.fit(fmri_filenames, y)
        return self

    def transform(self, X_df):
        fmri_filenames = X_df['fmri_msdl']
        X_connectome = self.transformer_fmri.transform(fmri_filenames)
        X_connectome = pd.DataFrame(X_connectome, index=X_df.index)
        X_connectome.columns = ['connectome_{}'.format(i)
                                for i in range(X_connectome.columns.size)]
        # get the anatomical information
        X_anatomy = X_df[[col for col in X_df.columns
                          if col.startswith('anatomy')]]
        X_anatomy = X_anatomy.drop(columns='anatomy_select')
        # concatenate both matrices
        return pd.concat([X_connectome, X_anatomy], axis=1)

class Classifier(BaseEstimator):
    def __init__(self):
        self.clf = RandomForestClassifier(n_estimators=100, n_jobs=-1)

    def fit(self, X, y):
        self.clf.fit(X, y)
        self.classes_ = unique_labels(y)
        return self
    
    def predict(self, X):
        return self.clf.predict(X)

    def predict_proba(self, X):
        return self.clf.predict_proba(X)

class Classifier(BaseEstimator):
    def __init__(self):
        self.clf_connectome = make_pipeline(StandardScaler(),
                                            LogisticRegression(C=1.))
        self.clf_anatomy = make_pipeline(StandardScaler(),
                                         LogisticRegression(C=1.))
        self.meta_clf = LogisticRegression(C=1.)

    def fit(self, X, y):
        X_anatomy = X[[col for col in X.columns if col.startswith('anatomy')]]
        X_connectome = X[[col for col in X.columns
                          if col.startswith('connectome')]]
        train_idx, validation_idx = train_test_split(range(y.size),
                                                     test_size=0.33,
                                                     shuffle=True,
                                                     random_state=42)
        X_anatomy_train = X_anatomy.iloc[train_idx]
        X_anatomy_validation = X_anatomy.iloc[validation_idx]
        X_connectome_train = X_connectome.iloc[train_idx]
        X_connectome_validation = X_connectome.iloc[validation_idx]
        y_train = y[train_idx]
        y_validation = y[validation_idx]

        self.clf_connectome.fit(X_connectome_train, y_train)
        self.clf_anatomy.fit(X_anatomy_train, y_train)

        y_connectome_pred = self.clf_connectome.predict_proba(
            X_connectome_validation)
        y_anatomy_pred = self.clf_anatomy.predict_proba(
            X_anatomy_validation)

        self.meta_clf.fit(
            np.concatenate([y_connectome_pred, y_anatomy_pred], axis=1),
            y_validation)
        self.classes_ = unique_labels(y)
        return self
    
    def predict(self, X):
        X_anatomy = X[[col for col in X.columns if col.startswith('anatomy')]]
        X_connectome = X[[col for col in X.columns
                          if col.startswith('connectome')]]

        y_anatomy_pred = self.clf_anatomy.predict_proba(X_anatomy)
        y_connectome_pred = self.clf_connectome.predict_proba(X_connectome)

        return self.meta_clf.predict(
            np.concatenate([y_connectome_pred, y_anatomy_pred], axis=1))

    def predict_proba(self, X):
        X_anatomy = X[[col for col in X.columns if col.startswith('anatomy')]]
        X_connectome = X[[col for col in X.columns
                          if col.startswith('connectome')]]

        y_anatomy_pred = self.clf_anatomy.predict_proba(X_anatomy)
        y_connectome_pred = self.clf_connectome.predict_proba(X_connectome)

        return self.meta_clf.predict_proba(
            np.concatenate([y_connectome_pred, y_anatomy_pred], axis=1))

In [47]:
results_anat = evaluation(data, labels)
results_anat_abide = evaluation(data_abide, labels_abide)
results_anat_adhd = evaluation(data_adhd, labels_adhd)
results_anat_abide_adhd = evaluation_vv(data_abide, labels_abide, data_adhd, labels_adhd)
results_anat_adhd_abide = evaluation_vv(data_adhd, labels_adhd, data_abide, labels_abide)


print('Dataset              Train                  Test')
print('               ROC-AUC | Accuracy    ROC-AUC | Accuracy')
print("ADHD+ABIDE     {:.3f}     {:.3f}       {:.3f}     {:.3f}".format(np.mean(results_anat['train_roc_auc']), np.mean(results_anat['train_accuracy']), np.mean(results_anat['test_roc_auc']), np.mean(results_anat['test_accuracy'])))
print("ABIDE          {:.3f}     {:.3f}       {:.3f}     {:.3f}".format(np.mean(results_anat_abide['train_roc_auc']), np.mean(results_anat_abide['train_accuracy']), np.mean(results_anat_abide['test_roc_auc']), np.mean(results_anat_abide['test_accuracy'])))
print("ADHD           {:.3f}     {:.3f}       {:.3f}     {:.3f}".format(np.mean(results_anat_adhd['train_roc_auc']), np.mean(results_anat_adhd['train_accuracy']), np.mean(results_anat_adhd['test_roc_auc']), np.mean(results_anat_adhd['test_accuracy'])))
print("ABIDE on ADHD  {:.3f}     {:.3f}       {:.3f}     {:.3f}".format(np.mean(results_anat_abide_adhd[1]), np.mean(results_anat_abide_adhd[0]), np.mean(results_anat_abide_adhd[3]), np.mean(results_anat_abide_adhd[2])))
print("ADHD on ABIDE  {:.3f}     {:.3f}       {:.3f}     {:.3f}".format(np.mean(results_anat_adhd_abide[1]), np.mean(results_anat_adhd_abide[0]), np.mean(results_anat_adhd_abide[3]), np.mean(results_anat_adhd_abide[2])))


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  3.3min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  2.2min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  1.1min finished


Dataset              Train                  Test
               ROC-AUC | Accuracy    ROC-AUC | Accuracy
ADHD+ABIDE     0.893     0.811       0.662     0.662
ABIDE          0.915     0.855       0.649     0.649
ADHD           0.918     0.730       0.598     0.598
ABIDE on ADHD  0.808     0.810       0.547     0.595
ADHD on ABIDE  0.615     0.721       0.506     0.529


## SVM 

In [39]:
ATLAS = ('msdl', 'basc064', 'basc122', 'basc197','harvard_oxford_cort_prob_2mm', 'craddock_scorr_mean','power_2011')

def _load_fmri(fmri_filenames):
    return np.array([
        pd.read_csv(subject_filename, header=None).values
        for subject_filename in fmri_filenames
    ])


class FeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.transformer_fmri_dict = {
            key: make_pipeline(
                FunctionTransformer(func=_load_fmri, validate=False),
                ConnectivityMeasure(kind='tangent', vectorize=True))
            for key in ATLAS
        }

    def fit(self, X_df, y):
        for atlas_name in self.transformer_fmri_dict.keys():
            atlas_col_name = 'fmri_' + atlas_name
            fmri_filename = X_df[atlas_col_name]
            self.transformer_fmri_dict[atlas_name].fit(fmri_filename, y)

        return self

    def transform(self, X_df):
        X_anatomy = X_df[[
            col for col in X_df.columns if col.startswith('anatomy')
        ]]
        X_anatomy = X_anatomy.drop(columns='anatomy_select')

        X_anatomy_column = X_anatomy.columns
        X_anatomy_index = X_df.index

        min_max_scaler = preprocessing.MinMaxScaler()
        X_anatomy_data = min_max_scaler.fit_transform(X_anatomy)
        X_anatomy = pd.DataFrame(
            data=X_anatomy_data,
            index=X_anatomy_index,
            columns=X_anatomy_column)

        X_atlas_df = pd.DataFrame(index=X_df.index)
        for atlas_name in self.transformer_fmri_dict.keys():
            atlas_col_name = 'fmri_' + atlas_name
            fmri_filename = X_df[atlas_col_name]

            X_connectome = self.transformer_fmri_dict[atlas_name].transform(
                fmri_filename)
            X_connectome = pd.DataFrame(X_connectome, index=X_df.index)
            X_connectome.columns = [
                atlas_name + '_connectome_{}'.format(i)
                for i in range(X_connectome.columns.size)
            ]

            X_anatomy.columns = [
                atlas_name + '_' + col for col in X_anatomy_column
            ]

            X_atlas_df = pd.concat([X_atlas_df, X_anatomy, X_connectome],
                                   axis=1)

        return X_atlas_df

ATLAS = ('msdl', 'basc064', 'basc122', 'basc197','harvard_oxford_cort_prob_2mm', 'craddock_scorr_mean','power_2011')

class Classifier(BaseEstimator):
    def __init__(self):
        self.base_clf_dict = {key: SVC(probability=True) for key in ATLAS}
        self.clf = LogisticRegression(C=1.)
        self.svc_parameters = [{
            'kernel': ['rbf'],
            'gamma': [1e-3, 1e-4],
            'C': [0.5, 1, 10, 100, 1000]
        }, {
            'kernel': ['linear'],
            'C': [0.5, 1, 10, 100, 1000]
        }]

    def _clf_data(self, X):
        X_atlas_dict = {
            key: X[[col for col in X.columns if col.startswith(key)]].values
            for key in ATLAS
        }
        X_meta_clf = None
        for key in self.base_clf_dict.keys():
            base_predict_pro = self.base_clf_dict[key].predict_proba(
                X_atlas_dict[key])

            if X_meta_clf is None:
                X_meta_clf = base_predict_pro
            else:
                X_meta_clf = np.concatenate([X_meta_clf, base_predict_pro],
                                            axis=1)

        return X_meta_clf

    def _grid_search(self, estimator, parameters, X, y):
        grid_search = GridSearchCV(estimator, parameters, n_jobs=-1, verbose=1)
        grid_search.fit(X, y)

        return grid_search.best_params_

    def fit(self, X, y):
        X_atlas_dict = {
            key: X[[col for col in X.columns if col.startswith(key)]].values
            for key in ATLAS
        }
        for key, val in X_atlas_dict.items():
            best_params = self._grid_search(self.base_clf_dict[key],
                                            self.svc_parameters, val, y)
            self.base_clf_dict[key].set_params(**best_params)

            self.base_clf_dict[key].fit(val, y)

        X_clf = self._clf_data(X)
        self.clf.fit(X_clf, y)
        self.classes_ = unique_labels(y)
        return self

    def predict(self, X):
        X_clf = self._clf_data(X)

        return self.clf.predict(X_clf)

    def predict_proba(self, X):
        X_clf = self._clf_data(X)

        return self.clf.predict_proba(X_clf)

In [ ]:
results_anat = evaluation(data, labels)
results_anat_abide = evaluation(data_abide, labels_abide)
results_anat_adhd = evaluation(data_adhd, labels_adhd)
results_anat_abide_adhd = evaluation_vv(data_abide, labels_abide, data_adhd, labels_adhd)
results_anat_adhd_abide = evaluation_vv(data_adhd, labels_adhd, data_abide, labels_abide)


print('Dataset              Train                  Test')
print('               ROC-AUC | Accuracy    ROC-AUC | Accuracy')
print("ADHD+ABIDE     {:.3f}     {:.3f}       {:.3f}     {:.3f}".format(np.mean(results_anat['train_roc_auc']), np.mean(results_anat['train_accuracy']), np.mean(results_anat['test_roc_auc']), np.mean(results_anat['test_accuracy'])))
print("ABIDE          {:.3f}     {:.3f}       {:.3f}     {:.3f}".format(np.mean(results_anat_abide['train_roc_auc']), np.mean(results_anat_abide['train_accuracy']), np.mean(results_anat_abide['test_roc_auc']), np.mean(results_anat_abide['test_accuracy'])))
print("ADHD           {:.3f}     {:.3f}       {:.3f}     {:.3f}".format(np.mean(results_anat_adhd['train_roc_auc']), np.mean(results_anat_adhd['train_accuracy']), np.mean(results_anat_adhd['test_roc_auc']), np.mean(results_anat_adhd['test_accuracy'])))
print("ABIDE on ADHD  {:.3f}     {:.3f}       {:.3f}     {:.3f}".format(np.mean(results_anat_abide_adhd[1]), np.mean(results_anat_abide_adhd[0]), np.mean(results_anat_abide_adhd[3]), np.mean(results_anat_abide_adhd[2])))
print("ADHD on ABIDE  {:.3f}     {:.3f}       {:.3f}     {:.3f}".format(np.mean(results_anat_adhd_abide[1]), np.mean(results_anat_adhd_abide[0]), np.mean(results_anat_adhd_abide[3]), np.mean(results_anat_adhd_abide[2])))
